<a href="https://colab.research.google.com/github/Visheshj123/AlbumCreation/blob/master/Pnemonia_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#install packages
!pip install numpy pydicom 
!pip install opencv-python
!pip install -Uqq fastbook

     |████████████████████████████████| 720 kB 14.1 MB/s 
     |████████████████████████████████| 1.2 MB 44.8 MB/s 
     |████████████████████████████████| 46 kB 3.2 MB/s 
     |████████████████████████████████| 189 kB 39.3 MB/s 
     |████████████████████████████████| 56 kB 4.4 MB/s 
     |████████████████████████████████| 51 kB 290 kB/s 


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import pydicom as dicom
import os
import cv2
import fastbook
fastbook.setup_book()
from fastbook import *
from fastai.vision.widgets import *

In [26]:
import pandas as pd

In [ ]:
!ls gdrive/MyDrive

 12go_booking_116196.pdf
 20161116_101654.jpg
'2018 GW NVC Semifinals Schedule (as of 3.21.18).pdf'
 234_183589.pdf
 4d4be3a21bda9c605eb72c59ec7e0a60.jpg
 94489607.pdf
 94490312.pdf
'Answer Key Practice exam 1.GIF'
 biochem.pdf
 BioPython.docx
'BioPython - vishy jajajaj.docx'
'Cardiac Lab Report.docx'
 Car_file_zip.zip
 CHEM11113graphsEnthalpy.docx
'Colab Notebooks'
 Colab_Notebooks
 commonapp21.docx
 commonapp21.gdoc
 ConditionsofContractEN.pdf
'Copy of semifinal NormaChain.pptx'
 DC_ToDo.gslides
"DD's edits.docx"
'Diagnostic Report_VJ.pdf'
 EmiratesETicket1.PDF
 EmiratesFareConditions.PDF
'First Night Waiver.docx'
'FullSizeR (1).jpg'
 FullSizeR.jpg
'gw schedule .docx'
 Her
 IMG_0137.jpg
 IMG_0600.JPG
 IMG_0821.jpg
 IMG_0998.HEIC
 IMG_0999.HEIC
 IMG_1001.HEIC
 IMG_1002.HEIC
 IMG_1003.HEIC
 IMG_1004.HEIC
 IMG_1005.HEIC
 IMG_1006.HEIC
 IMG_1007.HEIC
 IMG_1008.HEIC
 IMG_1009.HEIC
 IMG_1010.HEIC
 IMG_1011.HEIC
 IMG_1012.HEIC
 IMG_1013.HEIC
 IMG_1014.HEIC
 IMG_1928.jpg
 IMG_3057.jpg
 IMG_3

In [ ]:
!unzip /content/gdrive/MyDrive/rsna-pneumonia-detection-challenge.zip


Streaming output truncated to the last 5000 lines.
  inflating: rsna-pneumonia-detection-challenge/stage_2_train_images/f868de0f-be9e-4da2-aee7-b0bc5b66e086.dcm  
  inflating: __MACOSX/rsna-pneumonia-detection-challenge/stage_2_train_images/._f868de0f-be9e-4da2-aee7-b0bc5b66e086.dcm  
  inflating: rsna-pneumonia-detection-challenge/stage_2_train_images/bfdbe64f-1ab9-4b72-a4b5-3ef6a057add2.dcm  
  inflating: __MACOSX/rsna-pneumonia-detection-challenge/stage_2_train_images/._bfdbe64f-1ab9-4b72-a4b5-3ef6a057add2.dcm  
  inflating: rsna-pneumonia-detection-challenge/stage_2_train_images/4dbc44ca-694a-4814-9a24-bc60c8d8d611.dcm  
  inflating: __MACOSX/rsna-pneumonia-detection-challenge/stage_2_train_images/._4dbc44ca-694a-4814-9a24-bc60c8d8d611.dcm  
  inflating: rsna-pneumonia-detection-challenge/stage_2_train_images/eb056e61-98f0-418d-a5e9-b82bd8d2db93.dcm  
  inflating: __MACOSX/rsna-pneumonia-detection-challenge/stage_2_train_images/._eb056e61-98f0-418d-a5e9-b82bd8d2db93.dcm  
  inflati

In [ ]:
#we will use our training set as our train, validation, and test set. Note there are multiple images belonging to the patient ID, therefore
img = dicom.dcmread("/content/rsna-pneumonia-detection-challenge/stage_2_train_images/0004cfab-14fd-4e49-80ba-63a80b6bddd6.dcm")

we will use our training set as our train, validation, and test set. Note there are multiple images belonging to the patient ID, therefore we will ensure that no patient_ids end up in multiple datasets


In [ ]:
img.pixel_array

array([[144, 128, 114, ..., 176, 184, 117],
       [125, 109,  95, ..., 171, 178, 111],
       [104,  89,  75, ..., 166, 175, 108],
       ...,
       [ 16,  14,  11, ...,  19,  15,   3],
       [ 18,  16,  12, ...,  21,  17,   5],
       [ 19,  17,  13, ...,  23,  19,   6]], dtype=uint8)

In [27]:
labels = pd.read_csv("/content/rsna-pneumonia-detection-challenge/stage_2_train_labels.csv")

In [28]:
labels.head()

patientId      x      y  width  height  Target
0  0004cfab-14fd-4e49-80ba-63a80b6bddd6    NaN    NaN    NaN     NaN       0
1  00313ee0-9eaa-42f4-b0ab-c148ed3241cd    NaN    NaN    NaN     NaN       0
2  00322d4d-1c29-4943-afc9-b6754be640eb    NaN    NaN    NaN     NaN       0
3  003d8fa0-6bf1-40ed-b54c-ac657f8495c5    NaN    NaN    NaN     NaN       0
4  00436515-870c-4b36-a041-de91049b9ab4  264.0  152.0  213.0   379.0       1

In [39]:
#convert diacome images to png
dest_path = "/content/gdrive/MyDrive/RSNA_jpg_dir/"
def diacom_to_jpg(path):
  dir_list = os.listdir(path)
  for img_path in dir_list:
    ds = dicom.dcmread(path + '/' + img_path)
    #create name with png extension 
    img_name = os.path.basename(img_path).replace('.dcm', '.jpg')
    pid = img_name.split(".")[0]
    label = labels.loc[labels["patientId"] == pid]['Target'][0]
    cv2.imwrite(dest_path + img_name + "_" + label, ds.pixel_array)
diacom_to_jpg("/content/rsna-pneumonia-detection-challenge/stage_2_train_images")

KeyError: ignored

In [29]:
labels.shape

(30227, 6)

In [ ]:
def get_label(o):
  name = os.path.basename


In [ ]:
pnemonia = DataBlock(
    blocks=(ImageBlock, CategoryBlock),
    get_items=get_image_files, 
    splitter=RandomSplitter(valid_pct=0.2, seed=42), 
    
)

'0004cfab-14fd-4e49-80ba-63a80b6bddd6.jpg'

In [34]:
y = labels.loc[labels["patientId"] == '0004cfab-14fd-4e49-80ba-63a80b6bddd6']['Target'][0]

In [38]:
y[0]

0